# Loading CMU-MOSI dataset

In [ ]:
!git clone https://github.com/pliang279/MultiBench.git

Cloning into 'MultiBench'...
remote: Enumerating objects: 6943, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 6943 (delta 72), reused 121 (delta 60), pack-reused 6789
Receiving objects: 100% (6943/6943), 51.07 MiB | 18.68 MiB/s, done.
Resolving deltas: 100% (4258/4258), done.


In [ ]:
%cd MultiBench

/content/MultiBench


In [1]:
!mkdir data
!pip install gdown && gdown https://drive.google.com/u/0/uc?id=1szKIqO0t3Be_W91xvf6aYmsVVUa7wDHU

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try apt install
    python3-xyz, where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Debian-packaged Python package,
    create a virtual environment using python3 -m venv path/to/venv.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip. Make
    sure you have python3-full installed.
    
    If you wish to install a non-Debian packaged Python application,
    it may be easiest to use pipx install xyz, which will manage a
    virtual environment for you. Make sure you have pipx installed.
    
    See /usr/share/doc/python3.11/README.venv for more information.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detai

In [2]:
import torch
import sys
import os

In [13]:
# Import the associated dataloader for affect datasets, which MOSI is a part of.
from datasets.affect.get_data import get_dataloader

# Create the training, validation, and test-set dataloaders
traindata, validdata, testdata = get_dataloader(
    '/mosi_raw.pkl', robust_test=False, max_pad=True, data_type='mosi', max_seq_len=50)

ImportError: dynamic module does not define module export function (PyInit__torchtext)

In [4]:
print(len(traindata)+len(validdata)+len(testdata))
print(len(traindata))


NameError: name 'traindata' is not defined

The first dimension is often associated with the batch size (32 samples).

The second dimension is related to the sequence length (50 time steps).

The third dimension represents the number of features or dimensions in each element of the sequence (300 features)

In [9]:
for batch in traindata:
  audio,video,text,labels=batch

  break

NameError: name 'traindata' is not defined

each word is stored as GloVe embedding

basically it means i have a batch size of 32 objects , each object has 50 words and each word is represented as vector of 300 features (GloVe embedding)

In [8]:
print(type(traindata))

NameError: name 'traindata' is not defined

# Extracting the text and labels as torch.tensor objects and storing as torch dataloader to give model

In [6]:
from torch.utils.data import TensorDataset,DataLoader,ConcatDataset

In [7]:
all_text = []
all_labels = []

for batch in traindata:
    text, labels = batch[-2:]  #text and labels are the last two components
    all_text.append(text)
    all_labels.append(labels)

# Concatenate all batches
all_text = torch.cat(all_text, dim=0)
all_labels = torch.cat(all_labels, dim=0)

# Create a new DataLoader with all text and labels
new_dataset = TensorDataset(all_text, all_labels)
train_text_dataloader = DataLoader(new_dataset, batch_size=32,shuffle=True)

NameError: name 'traindata' is not defined

In [ ]:
print(len(train_text_dataloader))

41


In [ ]:
for i in train_text_dataloader:
  a,b=i
  print(b.shape)
  print(type(b))
  break

torch.Size([32, 1])
<class 'torch.Tensor'>


In [ ]:
all_text = []
all_labels = []

for batch in validdata:
    text, labels = batch[-2:]  # text and labels are the last two components
    all_text.append(text)
    all_labels.append(labels)

# Concatenate all batches
all_text = torch.cat(all_text, dim=0)
all_labels = torch.cat(all_labels, dim=0)

# Create a new DataLoader with all text and labels
new_dataset = TensorDataset(all_text, all_labels)
val_text_dataloader = DataLoader(new_dataset, batch_size=32,shuffle=True)

In [ ]:
all_text = []
all_labels = []

for batch in testdata:
    text, labels = batch[-2:]  # Assuming text and labels are the last two components
    all_text.append(text)
    all_labels.append(labels)

# Concatenate all batches
all_text = torch.cat(all_text, dim=0)
all_labels = torch.cat(all_labels, dim=0)

# Create a new DataLoader with all text and labels
new_dataset = TensorDataset(all_text, all_labels)
test_text_dataloader = DataLoader(new_dataset, batch_size=32,shuffle=True)

In [ ]:
for a,b in test_text_dataloader:
  print(b)
  b=b.unsqueeze(1).repeat(1, 50, 1)
  print("-------------")
  print(b)
  break

# Creating text model (unimodel) architecture from paper for sentiment regression

In [ ]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter

class LanguageModel(nn.Module):
    def __init__(self,input_size, hidden_size, num_layers,fc1_size, fc2_size,si_size):
        super(LanguageModel, self).__init__()

        self.output_range = Parameter(torch.FloatTensor([6]), requires_grad=False)
        self.output_shift = Parameter(torch.FloatTensor([-3]), requires_grad=False)

        # LSTM layer (stacked LSTM)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,batch_first=True)

        # Fully connected layers

        #fc1 gets hidden_size dimension values as input
        self.fc1 = nn.Linear(hidden_size, fc1_size)
        self.fc2 = nn.Linear(fc1_size, fc2_size)
        self.si1=nn.Linear(fc2_size,si_size)
        self.si2=nn.Linear(si_size,si_size)

        self.dropout=nn.Dropout(p=0.15)


        # Output layer
        self.output_layer = nn.Linear(fc2_size, 1)

        # Activation functions
        self.relu = nn.ReLU()
        self.sigmoid=nn.Sigmoid()

    def forward(self, x):
        # x = x.view(-1, x.size(-1), x.size(-2))

        # LSTM layer

        lstm_out, (hidden_states, cell_states) = self.lstm(x)

        fc1_out = self.relu(self.fc1(hidden_states.squeeze()))
        drop = self.dropout(fc1_out)
        fc2_out = self.relu(self.fc2(drop))


        si1=self.relu(self.si1(fc2_out))
        si2=self.relu(self.si2(si1))

        # Output layer with Sigmoid activation
        output = self.sigmoid(self.output_layer(si2))
        #get output between -3 and +3
        output=output*self.output_range+self.output_shift

        return output

## **Hidden size parameter in a LSTM**

1) Independence: In principle, the hidden size does not need to be directly related to the input size. You could have a large hidden size with a small input size or vice versa. The hidden size determines the capacity of the LSTM to capture and store information across time steps, while the input size defines the dimensionality of the input data at each time step.

2) Task Complexity: The choice of the hidden size is often influenced by the complexity of the task you are working on. More complex tasks may benefit from a larger hidden size as it provides the model with more capacity to learn intricate patterns in the data.

3) Computational Resources: Larger hidden sizes come with increased computational requirements. Training models with larger hidden sizes may take more time and resources. Therefore, the choice of hidden size can also depend on the available computational resources.



## **num_layers parameter in LSTM:**

Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM, with the second LSTM taking in outputs of the first LSTM and computing the final results. Default: 1

## **Hidden state of a LSTM unit:**

Let's break down the components of the hidden state:

1) Memory of Previous Information:

The hidden state encapsulates information from previous time steps in a sequence.
It acts as a form of memory that allows the LSTM to retain information over long distances in the sequence, mitigating the vanishing gradient problem encountered in simpler RNN architectures.

2) Capture of Relevant Patterns:

The LSTM hidden state is designed to capture and store relevant patterns, dependencies, and context within the sequence.
It selectively updates and forgets information through its gating mechanisms (input gate, forget gate, and output gate), allowing it to focus on important elements and discard less relevant information.

3) Interaction with Current Input:

The hidden state influences the processing of the current input at the current time step.
It combines information from the previous hidden state with the information from the current input to produce an updated hidden state for the current time step.

4) Encoded Representation:

The hidden state serves as an encoded representation of the information learned from the entire history of the sequence processed up to the current time step.

In [ ]:
# model parameters

#LSTM parameters
input_size = 300 # since each word is in GloVe embedding format
hidden_size = 128 # hyperparameter for each LSTM layer. refers to the dimension of the hidden state value
num_layers=1

# following are values defined in paper
fc1_size = 128
fc2_size = 128
si_size=128

model = LanguageModel(input_size, hidden_size,num_layers, fc1_size, fc2_size,si_size)

print(model)


LanguageModel(
  (lstm): LSTM(300, 128, batch_first=True)
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (si1): Linear(in_features=128, out_features=128, bias=True)
  (si2): Linear(in_features=128, out_features=128, bias=True)
  (dropout): Dropout(p=0.15, inplace=False)
  (output_layer): Linear(in_features=128, out_features=1, bias=True)
  (relu): ReLU()
  (sigmoid): Sigmoid()
)


# Training the text model (unimodel)

In [ ]:
Loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(list(model.parameters())[2:],lr=0.00006)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.0003)
num_epochs = 150

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
for epoch in range(num_epochs):

    print("EPOCH : ",epoch+1)
    # Training
    total_train_loss=0.0
    num_sequences=0
    model.train()  # Set the model to training mode
    for inputs, targets in train_text_dataloader:
        # targets=targets.unsqueeze(1).repeat(1, 50, 1)
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)
        loss = Loss(outputs, targets)
        total_train_loss+=loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights
        num_sequences+=1

    average_train_loss = total_train_loss / num_sequences
    print("-------------Training----------------")
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {average_train_loss:.4f}')

    # Validation
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        total_val_loss = 0.0
        num_sequences=0
        val_all_predictions = []
        val_all_targets = []
        for val_inputs, val_targets in val_text_dataloader:
            val_outputs = model(val_inputs)
            val_loss = Loss(val_outputs, val_targets)
            total_val_loss += val_loss.item()

            # Calculate R2 score
            val_targets_np = val_targets.cpu().numpy()
            val_outputs_np = val_outputs.cpu().numpy()


            val_all_predictions.extend(val_outputs_np)
            val_all_targets.extend(val_targets_np)
            num_sequences+=1

        average_val_loss = total_val_loss / num_sequences
        r_squared = r2_score(val_all_targets, val_all_predictions)

        print("--------------Validation----------")
        print(f'Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {average_val_loss:.4f}, R2 Score: {r_squared:.4f}')



    print("\n \n")


EPOCH :  1
-------------Training----------------
Epoch [1/150], Loss: 2.3158
--------------Validation----------
Epoch [1/150], Validation Loss: 2.8059, R2 Score: -0.0763

 

EPOCH :  2
-------------Training----------------
Epoch [2/150], Loss: 2.3659
--------------Validation----------
Epoch [2/150], Validation Loss: 2.7682, R2 Score: -0.0555

 

EPOCH :  3
-------------Training----------------
Epoch [3/150], Loss: 2.3208
--------------Validation----------
Epoch [3/150], Validation Loss: 2.6769, R2 Score: -0.0383

 

EPOCH :  4
-------------Training----------------
Epoch [4/150], Loss: 2.3170
--------------Validation----------
Epoch [4/150], Validation Loss: 2.7116, R2 Score: -0.0275

 

EPOCH :  5
-------------Training----------------
Epoch [5/150], Loss: 2.3081
--------------Validation----------
Epoch [5/150], Validation Loss: 2.6107, R2 Score: -0.0259

 

EPOCH :  6
-------------Training----------------
Epoch [6/150], Loss: 2.2874
--------------Validation----------
Epoch [6/150], Val

In PyTorch, the unsqueeze(0) method is used to add a new dimension of size 1 at the specified position. In this case, it adds a new dimension at the beginning of the tensor targets. The resulting tensor has a size of (1, original_size).

The repeat(50, 1, 1) method is then applied to this tensor. This function creates a new tensor by repeating the original tensor along each specified dimension. In this case, it repeats the tensor 50 times along the first dimension, and once along the second and third dimensions.

So, if targets initially had a size of (original_size), after unsqueeze(0).repeat(50, 1, 1), the resulting tensor will have a size of (50, original_size). The values in the tensor will be the same across the repeated dimensions.

In [ ]:
from sklearn.metrics import r2_score

model.eval()

with torch.no_grad():
    all_predictions = []
    all_targets = []
    total_test_loss=0
    num_sequences=0
    for test_inputs, test_targets in test_text_dataloader:
        # test_targets=test_targets.unsqueeze(1).repeat(1, 50, 1)
        test_outputs = model(test_inputs)
        test_loss=Loss(test_outputs,test_targets)
        total_test_loss+=test_loss
        # Convert predictions and targets to numpy arrays
        predictions = test_outputs.numpy().flatten()
        targets = test_targets.numpy().flatten()

        all_predictions.extend(predictions)
        all_targets.extend(targets)
        num_sequences+=1

    mae=total_test_loss/num_sequences
    r_squared = r2_score(all_targets, all_predictions)

    print(f'Test MAE: {mae:.4f}, R-squared: {r_squared:.4f}')


Test MAE: 2.1018, R-squared: 0.1687


Test MAE: 1.0795, R-squared: 0.2011

# Saving the model:


In [ ]:
torch.save(model.state_dict(), '/content/drive/MyDrive/multi_model_SA/final_model_MAE-1.0795_R-0.2011.pth')